In [1]:
from pathlib import Path

if Path.cwd().stem == "features":
    %cd ../..
    %load_ext autoreload
    %autoreload 2

/Users/visser/drive/PhD/Code/pain-measurement


see https://canvas.ox.ac.uk/courses/214847/pages/2-dot-5-eeg-biomarkers-and-applications-of-machine-learning-aleksandra-vuckovic-12-dot-12-dot-23?module_item_id=2163271

In [2]:
import logging
import math
import os
from functools import wraps
from pathlib import Path

import holoviews as hv
import hvplot.polars
import matplotlib.pyplot as plt
import neurokit2 as nk
import numpy as np
import pandas as pd
import panel as pn
import plotly.express as px
import polars as pl
import scipy.signal as signal

from src.data.config_data import DataConfigBase
from src.data.config_data_raw import RAW_DICT, RAW_LIST, RawConfig
from src.data.config_participant import PARTICIPANT_LIST, ParticipantConfig
from src.data.make_dataset import load_dataset
from src.features.quality_checks import check_sample_rate
from src.features.transformations import (
    add_timedelta_column,
    interpolate,
    map_participant_datasets,
    map_trials,
    merge_datasets,
    scale_min_max,
    scale_standard,
)
from src.log_config import configure_logging
from src.visualization.plot_data import (
    plot_data_panel,
    plot_trial_matplotlib,
    plot_trial_plotly,
)

configure_logging(
    stream_level=logging.DEBUG,
    ignore_libs=["matplotlib", "Comm", "bokeh", "tornado", "param"],
)

pl.Config.set_tbl_rows(7)  # don't print too many rows in the book
plt.rcParams["figure.figsize"] = [15, 5]  # default is [6, 4]

In [3]:
participant_number = 0
modality = "eeg"
data_config = RAW_DICT[modality]
sampling_rate = data_config.sampling_rate

eeg_raw = load_dataset(PARTICIPANT_LIST[participant_number], RAW_DICT[modality]).dataset
check_sample_rate(eeg_raw)
check_sample_rate(eeg_raw, unique_timestamp=True)

eeg_raw = eeg_raw.unique("Timestamp").sort(
    "Timestamp"
)  # actually slightly faster than maintain_order=True
eeg_raw = add_timedelta_column(eeg_raw)
logging.warning("Working with unique timestamps.")
eeg_raw

13:15:34 | DEBUG   | make_dataset | Dataset 'eeg' for participant 005_leo loaded from data/raw/005_leo/005_leo_eeg.csv
13:15:34 | DEBUG   | quality_checks | Sample rate per trial: [499.99 499.98 499.98 499.97 499.98 499.99 499.99 500.   499.99 499.96
 499.98 499.98]
13:15:34 | INFO    | quality_checks | The mean sample rate is 499.98.
13:15:34 | INFO    | quality_checks | Checking sample rate for unique timestamps.
13:15:34 | DEBUG   | quality_checks | Sample rate per trial: [499.99 499.98 499.98 499.97 499.98 499.99 499.99 500.   499.99 499.96
 499.98 499.98]
13:15:34 | INFO    | quality_checks | The mean sample rate is 499.98.
13:15:34 | WARNING | root | Working with unique timestamps.


Timestamp,EEG_RAW_Ch1,EEG_RAW_Ch2,EEG_RAW_Ch3,EEG_RAW_Ch4,EEG_RAW_Ch5,EEG_RAW_Ch6,EEG_RAW_Ch7,EEG_RAW_Ch8,Trial,Time
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,duration[ms]
65974.137,1405.906006,1159.094971,-3066.920898,30849.507812,23720.505859,9430.075195,9421.158203,9413.195312,0.0,1m 5s 974ms
65977.4359,1405.715942,1160.764038,-3067.922119,30846.503906,23717.882812,9427.691406,9420.061523,9411.335938,0.0,1m 5s 977ms
65978.5813,1399.230957,1155.42395,-3074.50293,30840.351562,23712.351562,9420.443359,9411.764648,9405.947266,0.0,1m 5s 978ms
…,…,…,…,…,…,…,…,…,…,…
3.1588e6,1121.759033,-810.098022,1563.310059,30259.611328,20759.345703,10221.433594,10018.634766,15774.298828,11.0,52m 38s 826ms
3.1588e6,1123.571045,-816.630981,1563.739014,30247.404297,20775.701172,10217.620117,10016.060547,15775.205078,11.0,52m 38s 828ms
3.1588e6,1117.229004,-810.145996,1559.925049,30237.964844,20761.683594,10204.411133,10006.094727,15754.987305,11.0,52m 38s 832ms


In [4]:
trial = 1

eeg = eeg_raw.filter(pl.col("Trial") == trial).select(
    "Timestamp", "Time", "EEG_RAW_Ch1"
)
eeg.plot(x="Time", y="EEG_RAW_Ch1", kind="line", width=1000, height=300)

:Curve   [Time]   (EEG_RAW_Ch1)